In [56]:
import pandas as pd
import json
import sqlite3

In [57]:
conn = sqlite3.connect('miniproject.db')
cursor = conn.cursor()

In [58]:
customers=pd.read_csv("customers.csv")
accounts=pd.read_csv("accounts.csv")
transactions=pd.read_csv("transactions.csv")
branches=pd.read_csv("branches.csv")
loans=pd.read_json("loans.json")
support_tickets=pd.read_json("support_tickets.json")

In [59]:
customers

,customer_id,name,gender,age,city,account_type,join_date
0,C0001,Richard Wilson,M,65,Kimberlyburgh,Savings,2023-08-27
1,C0002,Holly Parker,F,41,Port David,Current,2023-12-08
2,C0003,Megan Alvarez,M,46,West Laurieton,Savings,2022-05-25
3,C0004,Nicholas Stewart,F,39,Amyville,Current,2020-11-30
4,C0005,Douglas Anderson,M,70,New Craigport,Current,2023-01-29
...,...,...,...,...,...,...,...
495,C0496,Jennifer Flores,M,50,Mullenland,Savings,2021-04-03
496,C0497,Melissa Martin,M,41,Shermanfurt,Current,2021-12-27
497,C0498,Christopher Johnson,M,69,Nataliemouth,Savings,2022-10-10
498,C0499,Alvin Compton,M,30,Millerstad,Current,2023-06-16


**CUSTOMERS**

In [60]:
cursor.execute('''CREATE TABLE IF NOT EXISTS customers (
                    customer_id VARCHAR(10) PRIMARY KEY,
                    name TEXT,
                    age INTEGER,
                    gender CHAR,
                    city VARCHAR(20),
                    account_type VARCHAR(10),
                    join_date DATE
                )''')

In [61]:
for _, row in customers.iterrows():
  cursor.execute("""
    INSERT OR IGNORE INTO customers(customer_id, name, age, gender, city, account_type, join_date)
    VALUES(?, ?, ?, ?, ?, ?, ?)
   """,(row['customer_id'],row['name'],row['age'],row['gender'],row['city'],row['account_type'],row['join_date']))
conn.commit()

In [62]:
cursor.execute("SELECT * FROM customers")
rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]

data = pd.DataFrame(rows,columns=column_names)
data

,customer_id,name,age,gender,city,account_type,join_date
0,C0001,Richard Wilson,65,M,Kimberlyburgh,Savings,2023-08-27
1,C0002,Holly Parker,41,F,Port David,Current,2023-12-08
2,C0003,Megan Alvarez,46,M,West Laurieton,Savings,2022-05-25
3,C0004,Nicholas Stewart,39,F,Amyville,Current,2020-11-30
4,C0005,Douglas Anderson,70,M,New Craigport,Current,2023-01-29
...,...,...,...,...,...,...,...
495,C0496,Jennifer Flores,50,M,Mullenland,Savings,2021-04-03
496,C0497,Melissa Martin,41,M,Shermanfurt,Current,2021-12-27
497,C0498,Christopher Johnson,69,M,Nataliemouth,Savings,2022-10-10
498,C0499,Alvin Compton,30,M,Millerstad,Current,2023-06-16


In [63]:
cursor.execute('''CREATE TABLE IF NOT EXISTS accounts (
                    customer_id VARCHAR(10) REFERENCES users2(customer_id),
                    account_balance FLOAT,
                    last_updated DATE
                )''')

In [64]:
for _, row in accounts.iterrows():
  cursor.execute("""
    INSERT INTO accounts(customer_id,account_balance,last_updated)
    VALUES(?, ?, ?)
   """,(row['customer_id'],row['account_balance'],row['last_updated']))
conn.commit()

In [65]:
cursor.execute("SELECT * FROM accounts")
rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]

data = pd.DataFrame(rows,columns=column_names)
data

,customer_id,account_balance,last_updated
0,C0001,293729.87,2025-05-24 09:46:29
1,C0002,150735.15,2025-05-24 09:46:29
2,C0003,170156.21,2025-05-24 09:46:29
3,C0004,311383.96,2025-05-24 09:46:29
4,C0005,60320.75,2025-05-24 09:46:29
...,...,...,...
995,C0496,149135.80,2025-05-24 09:46:29
996,C0497,376153.36,2025-05-24 09:46:29
997,C0498,320254.17,2025-05-24 09:46:29
998,C0499,484758.19,2025-05-24 09:46:29


**QUESTION 1**

In [66]:
cursor.execute('''
    SELECT
        customers.city,
        COUNT(DISTINCT customers.customer_id) AS total_customer_per_city,
        ROUND(AVG(accounts.account_balance),2) AS avg_balance
    FROM
        customers
    JOIN
        accounts  ON customers.customer_id = accounts.customer_id
    GROUP BY
        customers.city
    ORDER BY
        avg_balance DESC
''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,city,total_customer_per_city,avg_balance
0,Margaretchester,1,498739.49
1,Kinghaven,1,498317.41
2,Johnmouth,1,494779.97
3,East Robin,1,494699.91
4,Millerstad,1,484758.19
...,...,...,...
482,Port Dakota,1,5128.16
483,West Ryanshire,1,3001.59
484,Juliaview,1,2647.41
485,North Dalefort,1,1862.66


**QUESTION 2**

In [67]:
cursor.execute('''
    SELECT
      customers.account_type,
      SUM(accounts.account_balance) AS total_balance_by_type
    FROM
      customers
    JOIN
      accounts ON customers.customer_id = accounts.customer_id
    GROUP BY
      customers.account_type
    ORDER BY
      total_balance_by_type DESC
''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,account_type,total_balance_by_type
0,Savings,1.237610e+08
1,Current,1.232352e+08


**QUESTION 3**

In [68]:
cursor.execute('''
      SELECT
      customers.name, customers.customer_id, customers.account_type,
      SUM(accounts.account_balance) AS total_balance
      FROM
         customers
      JOIN
        accounts ON customers.customer_id = accounts.customer_id
      GROUP BY customers.name
      ORDER BY total_balance DESC LIMIT 10
''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,name,customer_id,account_type,total_balance
0,Anthony Ross,C0136,Current,1026991.26
1,Amy Nguyen,C0314,Current,997478.98
2,Carolyn Higgins,C0011,Savings,996634.82
3,Lisa Russell,C0111,Current,994081.20
4,Amber Williams,C0392,Savings,989559.94
5,Audrey Peck,C0077,Savings,989399.82
6,Alvin Compton,C0499,Current,969516.38
7,Sharon Bennett DVM,C0338,Savings,965492.74
8,Denise Campbell,C0317,Current,963493.16
9,Jeffrey Smith,C0220,Savings,963136.28


**QUESTION 4**

In [69]:
cursor.execute('''
    SELECT customers.customer_id, customers.name, customers.join_date, accounts.account_balance
    FROM
    customers
    JOIN
    accounts ON customers.customer_id= accounts.customer_id
    WHERE STRFTIME('%Y', customers.join_date) = '2023' AND accounts.account_balance > 100000
''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,customer_id,name,join_date,account_balance
0,C0001,Richard Wilson,2023-08-27,293729.87
1,C0002,Holly Parker,2023-12-08,150735.15
2,C0007,Jasmine Morris,2023-10-09,291736.90
3,C0017,Jesus Davenport,2023-06-07,345239.48
4,C0021,William Jones,2023-04-01,185053.68
...,...,...,...,...
221,C0478,Ryan Jackson,2023-04-18,400751.68
222,C0484,Rodney Cruz,2023-03-09,150039.17
223,C0489,Andrea Austin,2023-01-28,407686.39
224,C0499,Alvin Compton,2023-06-16,484758.19


**TRANSACTIONS**

In [70]:
cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
                    customer_id VARCHAR(10) REFERENCES customers(customer_id),
                    txn_id VARCHAR(10),
                    txn_type VARCHAR(10),
                    txn_time TIME,
                    amount FLOAT,
                    status VARCHAR(10)
               )''')

In [71]:
for _, row in transactions.iterrows():
  cursor.execute("""
    INSERT INTO transactions(customer_id,txn_id,txn_type,txn_time,amount,status)
    VALUES(?, ?, ?,?,?,?)
   """,(row['customer_id'],row['txn_id'],row['txn_type'],row['txn_time'],row['amount'],row['status']))
conn.commit()

In [72]:
cursor.execute("SELECT * FROM transactions")
rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]

data = pd.DataFrame(rows,columns=column_names)
data

,customer_id,txn_id,txn_type,txn_time,amount,status
0,C0363,T00001,deposit,2025-02-14 06:26:00,44442.71,success
1,C0432,T00002,deposit,2025-03-27 07:21:21,15737.59,success
2,C0084,T00003,withdrawal,2024-07-31 18:28:06,82150.70,success
3,C0067,T00004,withdrawal,2024-11-11 12:18:47,20313.86,success
4,C0485,T00005,withdrawal,2025-04-15 10:52:59,74536.08,success
...,...,...,...,...,...,...
19995,C0260,T09996,withdrawal,2024-07-28 08:41:01,54743.72,success
19996,C0172,T09997,purchase,2024-06-23 07:24:30,13087.77,failed
19997,C0494,T09998,withdrawal,2025-02-10 01:47:28,66846.76,failed
19998,C0499,T09999,withdrawal,2024-12-11 04:45:07,90759.88,success


**QUESTION 5**

In [73]:
cursor.execute('''
    SELECT transactions.txn_type,
    SUM(transactions.amount) AS total_transction_volume
    FROM customers
    JOIN transactions ON customers.customer_id=transactions.customer_id
    GROUP BY Txn_type
    ORDER BY total_transction_volume DESC
''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,txn_type,total_transction_volume
0,deposit,3.951497e+08
1,withdrawal,3.027746e+08
2,purchase,1.476681e+08
3,transfer,1.011856e+08
4,online fraud,4.765900e+07


**QUESTION 6**

In [74]:
cursor.execute('''
      SELECT txn_ID, Customer_ID, txn_time, status
      FROM transactions
      WHERE status = 'failed'
      ''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,txn_id,customer_id,txn_time,status
0,T00010,C0139,2025-03-17 03:09:38,failed
1,T00014,C0413,2024-11-19 17:08:50,failed
2,T00018,C0316,2024-10-09 12:24:21,failed
3,T00020,C0493,2025-01-17 00:32:06,failed
4,T00032,C0029,2024-07-25 23:26:09,failed
...,...,...,...,...
2989,T09992,C0193,2025-02-03 08:39:42,failed
2990,T09994,C0221,2025-02-14 10:55:12,failed
2991,T09997,C0172,2024-06-23 07:24:30,failed
2992,T09998,C0494,2025-02-10 01:47:28,failed


**QUESTION 7**

In [75]:
cursor.execute('''
      SELECT customer_ID,
      SUM(amount) AS Total_Transaction_amount
      FROM transactions
      GROUP BY customer_ID
      ''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,customer_id,Total_Transaction_amount
0,C0001,2156280.56
1,C0002,1325942.88
2,C0003,1728926.10
3,C0004,1470280.36
4,C0005,1257666.10
...,...,...
495,C0496,3408705.36
496,C0497,2191784.52
497,C0498,1603635.96
498,C0499,2463372.68


**QUESTION 8**

In [76]:
cursor.execute('''
      SELECT amount,
      txn_ID
      FROM transactions
      WHERE amount >= 20000
      ''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,amount,txn_id
0,44442.71,T00001
1,82150.70,T00003
2,20313.86,T00004
3,74536.08,T00005
4,85463.28,T00006
...,...,...
15919,57358.56,T09995
15920,54743.72,T09996
15921,66846.76,T09998
15922,90759.88,T09999


**BRANCHES**

In [77]:
cursor.execute('''CREATE TABLE IF NOT EXISTS Branches (
                    Branch_ID INTEGER,
                    Branch_Name VARCHAR(50),
                    City VARCHAR(20),
                    Manager_Name VARCHAR(20),
                    Total_Employees INTEGER,
                    Branch_Revenue FLOAT,
                    Opening_Date DATE,
                    Performing_Rating INTEGER
               )''')

In [78]:
for _, row in branches.iterrows():
  cursor.execute("""
    INSERT INTO Branches(Branch_ID, Branch_Name, City, Manager_Name, Total_Employees, Branch_Revenue, Opening_Date, Performing_Rating)
    VALUES(?, ?, ?, ?, ?, ?, ?, ?)
   """,(row['Branch_ID'], row['Branch_Name'], row['City'], row['Manager_Name'], row['Total_Employees'], row['Branch_Revenue'], row['Opening_Date'], row['Performance_Rating']))
conn.commit()


In [79]:
cursor.execute("SELECT * FROM Branches")
rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]

data = pd.DataFrame(rows,columns=column_names)
data

,Branch_ID,Branch_Name,City,Manager_Name,Total_Employees,Branch_Revenue,Opening_Date,Performing_Rating
0,1,"Wang, Mcdonald and Church Branch",Kellitown,Kevin Jones,141,16849288.09,2008-08-01,2
1,2,Crawford-Torres Branch,Angelaburgh,Andrea Steele,39,5822279.32,2015-09-17,1
2,3,Bridges Inc Branch,Johnsonfort,Michael Coleman,195,2114594.03,2012-07-30,3
3,4,Garcia Group Branch,New Kyleberg,Dawn Cooper,144,3390389.63,2018-06-27,2
4,5,"Arroyo, Moore and Freeman Branch",West Coryborough,Samantha Kennedy,42,2771910.11,2012-01-04,5
...,...,...,...,...,...,...,...,...
1555,516,Lowery and Sons Branch,Lake Charles,Daniel Hunt,130,6562890.41,2007-06-22,4
1556,517,Brooks Group Branch,New Christina,Rodney Browning,19,19078758.01,2021-07-20,1
1557,518,Harris-Marshall Branch,Emilymouth,Lorraine Johnson,177,2801634.32,2011-08-18,5
1558,519,"Meadows, Taylor and Butler Branch",Port Michael,David Smith,76,17653138.31,2006-05-21,4


**LOANS**

In [94]:
loans = pd.read_json('loans.json')

In [80]:
cursor.execute('''CREATE TABLE IF NOT EXISTS loans (
                    Loan_ID INTEGER,
                    Customer_ID INTEGER,
                    Account_ID INTEGER,
                    Branch VARCHAR(20),
                    Loan_Type VARCHAR(20),
                    Loan_Amount FLOAT,
                    Interest_Rate FLOAT,
                    Loan_Term_Months INTEGER,
                    Start_Date DATE,
                    End_Date DATE,
                    Loan_Status VARCHAR(10)
               )''')

In [81]:
for _, row in loans.iterrows():
  cursor.execute("""
    INSERT INTO loans(Loan_ID, Customer_ID, Account_ID, Branch, Loan_Type, Loan_Amount, Interest_Rate, Loan_Term_Months, Start_Date, End_Date, Loan_Status)
    VALUES(?, ?, ?,?,?,?, ? ,? ,?, ?, ?)
   """,(row['Loan_ID'], row['Customer_ID'], row['Account_ID'], row['Branch'], row['Loan_Type'], row['Loan_Amount'], row['Interest_Rate'], row['Loan_Term_Months'], row['Start_Date'], row['End_Date'], row['Loan_Status']))
conn.commit()

In [82]:
cursor.execute("SELECT * FROM loans")
rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]

data = pd.DataFrame(rows,columns=column_names)
data

,Loan_ID,Customer_ID,Account_ID,Branch,Loan_Type,Loan_Amount,Interest_Rate,Loan_Term_Months,Start_Date,End_Date,Loan_Status
0,1,1559,2090,Perez Ltd Branch,Personal,259805.0,12.43,60,2017-07-03,2022-06-07,Active
1,2,1028,2575,Harris-Marshall Branch,Business,3589336.0,6.75,24,2017-06-14,2019-06-04,Active
2,3,1164,2433,"Taylor, David and Brown Branch",Auto,4621300.0,9.86,240,2021-08-10,2041-04-27,Closed
3,4,1100,2368,Robertson and Sons Branch,Home,2380953.0,7.74,120,2016-02-23,2026-01-01,Approved
4,5,1081,2566,"Parrish, Hall and Adams Branch",Home,2268961.0,12.96,240,2021-01-05,2040-09-22,Approved
...,...,...,...,...,...,...,...,...,...,...,...
1101,549,1534,2214,"Potts, Gutierrez and Hancock Branch",Education,2123346.0,12.40,12,2017-03-26,2018-03-21,Defaulted
1102,550,1563,2534,"Cunningham, Lee and Fleming Branch",Education,3314199.0,11.95,84,2018-08-16,2025-07-10,Active
1103,551,1224,2116,Richardson-Morales Branch,Personal,2174941.0,10.35,84,2017-03-24,2024-02-16,Closed
1104,552,1406,2249,"Ross, Hayes and Erickson Branch",Auto,3692011.0,9.10,36,2023-11-28,2026-11-12,Closed


**QUESTION 9**

In [83]:
cursor.execute('''
    SELECT Interest_Rate, Loan_Type,
    ROUND(AVG(Loan_Amount),2) AS avg_loan_amount
    FROM loans
    GROUP BY Loan_Type

''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,Interest_Rate,Loan_Type,avg_loan_amount
0,9.86,Auto,2399260.98
1,6.75,Business,2793968.87
2,11.27,Education,2393947.70
3,7.74,Home,2344576.66
4,12.43,Personal,2461543.75


**QUESTION 10**

In [84]:
cursor.execute('''
      SELECT Customer_ID, Loan_ID, Loan_Status
      FROM loans
      WHERE Loan_Status = 'Active' OR Loan_Status = 'Approved'
      ''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,Customer_ID,Loan_ID,Loan_Status
0,1559,1,Active
1,1028,2,Active
2,1100,4,Approved
3,1081,5,Approved
4,1082,6,Active
...,...,...,...
565,1191,544,Approved
566,1093,545,Approved
567,1255,548,Approved
568,1563,550,Active


**QUESTION 11**

In [85]:
cursor.execute('''
      SELECT Customer_ID, Loan_ID, Loan_Amount, Loan_Status
      FROM loans
      WHERE Loan_Status == 'Defaulted'
      ORDER BY Loan_Amount DESC LIMIT 5
      ''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,Customer_ID,Loan_ID,Loan_Amount,Loan_Status
0,1510,201,4888307.0,Defaulted
1,1510,201,4888307.0,Defaulted
2,1533,46,4886771.0,Defaulted
3,1533,46,4886771.0,Defaulted
4,1362,176,4873847.0,Defaulted


**CREDIT CARD**

In [95]:
credit_cards = pd.read_json('credit_cards.json')

In [96]:
credit_cards

,Card_ID,Customer_ID,Account_ID,Branch,Card_Number,Card_Type,Card_Network,Credit_Limit,Current_Balance,Issued_Date,Expiry_Date,Status
0,1,1559,2090,Perez Ltd Branch,4960013389083863,Business,Visa,178393,39819.23,2015-04-13,2019-10-27,Expired
1,2,1160,2221,Miller Inc Branch,6511615594078161,Platinum,RuPay,416026,175818.58,2015-01-27,2018-12-18,Expired
2,3,1297,2082,"Miles, Patterson and Murray Branch",6316475255341928,Gold,RuPay,419398,29171.29,2021-06-22,2025-07-19,Active
3,4,1333,2058,"Shepard, Gonzales and Cooper Branch",3305641395376724,Gold,Amex,385544,265316.71,2019-04-04,2023-10-07,Active
4,5,1371,2225,Wilson PLC Branch,6287101226916697,Platinum,RuPay,274622,246561.92,2023-05-18,2029-08-26,Expired
...,...,...,...,...,...,...,...,...,...,...,...,...
552,553,1160,2599,"Huff, Bryant and Thomas Branch",6295966048736826,Silver,RuPay,286715,193389.66,2023-06-03,2028-03-08,Active
553,554,1163,2411,Morris-Prince Branch,3865369455158886,Gold,Amex,108003,61642.17,2017-06-13,2021-08-18,Blocked
554,555,1161,2182,"Robertson, Moore and Howell Branch",3640857216966630,Gold,Amex,200094,165305.34,2023-06-27,2029-01-31,Active
555,556,1333,2256,Green-Keith Branch,4297150968832753,Gold,Visa,433856,177349.60,2021-09-21,2027-12-02,Blocked


In [98]:
credit_cards.columns

Index(['Card_ID', 'Customer_ID', 'Account_ID', 'Branch', 'Card_Number',
       'Card_Type', 'Card_Network', 'Credit_Limit', 'Current_Balance',
       'Issued_Date', 'Expiry_Date', 'Status'],
      dtype='object')

In [112]:
cursor.execute('DROP TABLE IF EXISTS Credit_Cards')
cursor.execute('''CREATE TABLE IF NOT EXISTS Credit_Cards (
                    Card_ID INTEGER,
                    Customer_ID INTEGER,
                    Account_ID VARCHAR(20),
                    Branch VARCHAR(20),
                    Card_Number INTEGER,
                    Card_Type VARCHAR(20),
                    Card_Network VARCHAR(20),
                    Credit_Limit INTEGER,
                    Current_Balance INTEGER,
                    Issued_Date DATE,
                    Expiry_Date DATE,
                    Status VARCHAR(10)
               )''')

In [114]:
for _, row in credit_cards.iterrows():
  cursor.execute("""
    INSERT INTO Credit_Cards(Card_ID, Customer_ID, Account_ID, Branch, Card_Number, Card_Type, Card_Network, Credit_Limit, Current_Balance, Issued_Date, Expiry_Date, Status)
    VALUES(?, ?, ?,?,?,?, ? ,? ,?, ?, ?, ?)
   """,(row['Card_ID'], row['Customer_ID'], row['Account_ID'], row['Branch'], row['Card_Number'], row['Card_Type'], row['Card_Network'], row['Credit_Limit'], row['Current_Balance'], row['Issued_Date'], row['Expiry_Date'], row['Status']))
conn.commit()

**SUPPORT TICKETS**

In [86]:
support_tickets = pd.read_json('support_tickets.json')

In [87]:
cursor.execute('''CREATE TABLE IF NOT EXISTS support_tickets (
                    Ticket_ID INTEGER,
                    Customer_ID INTEGER,
                    Account_ID VARCHAR(20),
                    Loan_ID INTEGER,
                    Branch_Name VARCHAR(20),
                    Issue_Category VARCHAR(20),
                    Description TEXT,
                    Open_Date DATE,
                    Close_Date DATE,
                    Priority VARCHAR(10),
                    Status VARCHAR(10),
                    Resolution_Remarks VARCHAR(20),
                    Support_Agent VARCHAR(20),
                    Channel VARCHAR(20),
                    Customer_Rating INTEGER
               )''')

In [88]:
for _, row in support_tickets.iterrows():
  cursor.execute("""
    INSERT INTO support_tickets(Ticket_ID, Customer_ID, Account_ID, Loan_ID, Branch_Name, Issue_Category, Description, Open_Date, Close_Date, Priority, Status, Resolution_Remarks, Support_Agent, Channel, Customer_Rating)
    VALUES(?, ?, ?,?,?,?, ? ,? ,?, ?, ?, ?, ?, ?, ?)
   """,(row['Ticket_ID'], row['Customer_ID'], row['Account_ID'], row['Loan_ID'], row['Branch_Name'], row['Issue_Category'], row['Description'], row['Date_Opened'], row['Date_Closed'], row['Priority'], row['Status'], row['Resolution_Remarks'], row['Support_Agent'], row['Channel'], row['Customer_Rating']))
conn.commit()

In [89]:
cursor.execute("SELECT * FROM support_tickets")
rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]

data = pd.DataFrame(rows,columns=column_names)
data

,Ticket_ID,Customer_ID,Account_ID,Loan_ID,Branch_Name,Issue_Category,Description,Open_Date,Close_Date,Priority,Status,Resolution_Remarks,Support_Agent,Channel,Customer_Rating
0,T00001,C0478,A1825,,Gonzalez-Martin Branch,Loan Payment Delay,Client raised loan payment delay.,2017-09-30,2017-10-24,Low,Closed,Card replaced and activated.,Leena Apte,Email,1
1,T00002,C0443,A3583,,Baldwin and Sons Branch,Loan Foreclosure,Client raised loan foreclosure.,2015-04-19,2015-05-12,Critical,In Progress,Card replaced and activated.,Owen Mahajan,In-person,1
2,T00003,C0057,A2616,442,"Porter, Osborne and Mendez Branch",Interest Rate Dispute,Customer reported interest rate dispute.,2016-09-28,2016-10-23,High,Resolved,Loan schedule updated.,Wishi Chhabra,Email,4
3,T00004,C0301,A8786,,"Stafford, Oneal and Williams Branch",Mismatch in Account Statement,Client raised mismatch in account statement.,2019-03-31,2019-04-10,High,Closed,Pending customer verification.,Gabriel Mutti,Email,1
4,T00005,C0238,A3734,,Evans-Mercado Branch,KYC Update Required,System detected kyc update required.,2017-08-11,,Critical,Resolved,Pending customer verification.,Ekani Oza,In-person,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,T00596,C0303,A9594,,Johnson Group Branch,Account Frozen,Customer reported account frozen.,2015-03-08,2015-03-30,Medium,Escalated,Issue resolved successfully.,Chaitaly Vaidya,Mobile App,3
1196,T00597,C0480,A5144,,Clark Inc Branch,Interest Rate Dispute,Customer reported interest rate dispute.,2020-01-17,2020-01-26,Medium,In Progress,Issue resolved successfully.,Ekapad Jhaveri,In-person,1
1197,T00598,C0270,A7108,427,Murphy-Johnston Branch,Interest Rate Dispute,Client raised interest rate dispute.,2016-04-22,2016-05-13,Critical,Resolved,Loan schedule updated.,Advik Bhagat,In-person,4
1198,T00599,C0144,A4959,121,"Herrera, Rollins and Hill Branch",Card Not Working,Customer reported card not working.,2015-02-26,2015-03-25,Low,In Progress,Card replaced and activated.,Wriddhish Sarraf,Phone,2


**QUESTION 14**

In [90]:
cursor.execute('''
      SELECT
        Issue_Category,
        Ticket_ID,
        Customer_ID,
        Branch_Name,
        Open_Date,
        Close_Date,
        CAST(julianday(Close_Date) - julianday(Open_Date) AS INTEGER) AS NO_OF_DAYS_TAKEN
        FROM support_tickets
        ORDER BY NO_OF_DAYS_TAKEN DESC LIMIT 15


      ''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,Issue_Category,Ticket_ID,Customer_ID,Branch_Name,Open_Date,Close_Date,NO_OF_DAYS_TAKEN
0,Interest Rate Dispute,T00012,C0392,"Wang, Mcdonald and Church Branch",2020-06-24,2020-07-24,30
1,Cheque Bounce,T00084,C0427,"Taylor, Hopkins and Morales Branch",2023-03-19,2023-04-18,30
2,Mismatch in Account Statement,T00094,C0219,"Miller, Baker and Owen Branch",2015-06-27,2015-07-27,30
3,EMI Auto-debit Failed,T00099,C0246,Tapia-Flynn Branch,2023-06-01,2023-07-01,30
4,Mismatch in Account Statement,T00142,C0437,Morgan Inc Branch,2016-06-12,2016-07-12,30
5,Interest Rate Dispute,T00153,C0126,"Austin, Rogers and Ayala Branch",2020-10-12,2020-11-11,30
6,Account Closure Request,T00174,C0374,"Jones, Page and Kennedy Branch",2019-04-16,2019-05-16,30
7,Mismatch in Account Statement,T00206,C0016,Vaughn Ltd Branch,2020-12-28,2021-01-27,30
8,Card Not Working,T00222,C0493,Lozano-Wilson Branch,2023-05-31,2023-06-30,30
9,Unauthorized Transaction,T00260,C0208,Moore-Soto Branch,2016-08-28,2016-09-27,30


**QUESTION 15**

In [91]:
cursor.execute('''
      SELECT Support_Agent,
      Customer_Rating,
      CAST(julianday(Close_Date) - julianday(Open_Date) AS INTEGER) AS NO_OF_DAYS_TAKEN,
      Status
      FROM support_tickets
      WHERE Customer_Rating >= 4 AND Priority = 'High' AND Status = 'Closed'
      ORDER BY NO_OF_DAYS_TAKEN ASC LIMIT 10
      ''')
rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
data = pd.DataFrame(rows,columns=column_names)
data

,Support_Agent,Customer_Rating,NO_OF_DAYS_TAKEN,Status
0,Gaurika Pillay,5,NaN,Closed
1,Ranveer Roy,4,NaN,Closed
2,Chaitaly Vaidya,5,NaN,Closed
3,Gaurika Pillay,5,NaN,Closed
4,Ranveer Roy,4,NaN,Closed
5,Chaitaly Vaidya,5,NaN,Closed
6,Owen Mahajan,4,8.0,Closed
7,Owen Mahajan,4,8.0,Closed
8,Vansha Sood,5,16.0,Closed
9,Vansha Sood,5,16.0,Closed
